# Shortest Path
![title](img/ej2.png)

### EX2: Basic example SP
1. Try the same procedure with this example
1. Check the solution
1. Try changing the method to SIMPLEX
1. Get the shortest path

## Model definitions

In [1]:
import numpy as np
from scipy.optimize import linprog
import pandas as pd

from mis_utils import *

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.latex.repr = True

In [2]:
# Definiciones de constantes

node_names = np.array(('s', '2', '3', '4', '5', 't'))

# Balances: Sale un producto desde s y llega hasta t
beq = np.array([1, 0, 0, 0, 0, -1])

NN = np.array([[0, 1, 1, 0, 0, 0],
               [0, 0, 0, 1, 0, 1],
               [0, 0, 0, 0, 1, 0],
               [0, 0, 0, 0, 0, 1],
               [0, 0, 0, 0, 0, 1],
               [0, 0, 0, 0, 0, 0]])

# Matrices resultantes de NN a NA
Aeq, arc_idxs = nn2na(NN, node_names = node_names, show_results = True)

# Guardo los nombres de los nodo-arco-nodo posibles
nan_names = get_col_names(NN, node_names, as_numpy=True, sep = "->")

# Restricciones, l <= x <= u
# Entre 0 y 1 sería lo teóricamente correcto, aunque como hay un límite que como el modelo indica, es de un producto
min_bound = 0
max_bound = 1
bounds = tuple([(min_bound, max_bound) for arcs in range(0, Aeq.shape[1])])

[[0 1]
 [0 2]
 [1 3]
 [1 5]
 [2 4]
 [3 5]
 [4 5]]
Input: 
 [[0 1 1 0 0 0]
 [0 0 0 1 0 1]
 [0 0 0 0 1 0]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 0]]

Column names: s2-s3-24-2t-35-4t-5t

Output: 
 [[ 1  1  0  0  0  0  0]
 [-1  0  1  1  0  0  0]
 [ 0 -1  0  0  1  0  0]
 [ 0  0 -1  0  0  1  0]
 [ 0  0  0  0 -1  0  1]
 [ 0  0  0 -1  0 -1 -1]]



In [3]:
print("Node Node matrix:")
display(pd.DataFrame(data=NN, columns=node_names, index = node_names))

Node Node matrix:


,s,2,3,4,5,t
s,0,1,1,0,0,0
2,0,0,0,1,0,1
3,0,0,0,0,1,0
4,0,0,0,0,0,1
5,0,0,0,0,0,1
t,0,0,0,0,0,0


In [4]:
print("Node Arcs matrix")
pd.DataFrame(data=Aeq, columns=nan_names, index = node_names)

Node Arcs matrix


,s->2,s->3,2->4,2->t,3->5,4->t,5->t
s,1,1,0,0,0,0,0
2,-1,0,1,1,0,0,0
3,0,-1,0,0,1,0,0
4,0,0,-1,0,0,1,0
5,0,0,0,0,-1,0,1
t,0,0,0,-1,0,-1,-1


In [5]:
# Vector de costos
# Con el orden de los nombres de columnas, sacado de los resultados. Column names: s2-s3-24-2t-35-4t-5t
C = np.array([2, 1, 2, 5, 2, 1, 2])

In [6]:
resume_df = pd.DataFrame(bounds, index=nan_names, columns=['Min bound', 'Max bound'])
resume_df['Costs'] = C

In [7]:
resume_df

,Min bound,Max bound,Costs
s->2,0,1,2
s->3,0,1,1
2->4,0,1,2
2->t,0,1,5
3->5,0,1,2
4->t,0,1,1
5->t,0,1,2


In [8]:
# Resumen
print('## Optimizer inputs ## \n\n'
      'Cost vector: %s \n\n'
      'Columns: %s \n\n'
      'A_eq Node-Arc matrix:\n%s \n\n'
      'b_eq demand-supply vector: %s \n\n'
      'Bounds of each X arc variable: %s \n' % (C, nan_names, Aeq, beq, bounds))

## Optimizer inputs ## 

Cost vector: [2 1 2 5 2 1 2] 

Columns: ['s->2' 's->3' '2->4' '2->t' '3->5' '4->t' '5->t'] 

A_eq Node-Arc matrix:
[[ 1  1  0  0  0  0  0]
 [-1  0  1  1  0  0  0]
 [ 0 -1  0  0  1  0  0]
 [ 0  0 -1  0  0  1  0]
 [ 0  0  0  0 -1  0  1]
 [ 0  0  0 -1  0 -1 -1]] 

b_eq demand-supply vector: [ 1  0  0  0  0 -1] 

Bounds of each X arc variable: ((0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1)) 



# Resuelvo con el método por defecto (interior-point)

In [9]:
# Optimización
res = linprog(C, A_eq=Aeq, b_eq=beq, bounds=bounds, method='interior-point')

selarcs = get_selected_arcs(res.x.round().astype(int), nan_names)

/home/diego/.virtualenvs/OS11/lib/python3.6/site-packages/ipykernel_launcher.py:2: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  


In [10]:
print('## Results ##')
print('The raw solution will be: %s' % res.x.round().astype(int))
print('The arcs that make the shortest path will be (from, to): %s' % selarcs)
print('The minimum cost will be: %0.2f ' % res.fun)
print(res.message)

## Results ##
The raw solution will be: [1 0 1 0 0 1 0]
The arcs that make the shortest path will be (from, to): ['s->2', '2->4', '4->t']
The minimum cost will be: 5.00 
Optimization terminated successfully.


# Resuelvo con el método simplex

In [11]:
# Optimización
res_simplex = linprog(C, A_eq=Aeq, b_eq=beq, bounds=bounds, method='simplex')

selarcs_simplex = get_selected_arcs(res_simplex.x.round().astype(int), nan_names)

/home/diego/.virtualenvs/OS11/lib/python3.6/site-packages/ipykernel_launcher.py:2: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  


In [12]:
print('## Results ##')
print('The raw solution will be: %s' % res_simplex.x.round().astype(int))
print('The arcs that make the shortest path from s to t will be: %s' % selarcs_simplex)
print('The minimum cost will be: %0.2f ' % res_simplex.fun)
print(res_simplex.message)

## Results ##
The raw solution will be: [0 1 0 0 1 0 1]
The arcs that make the shortest path from s to t will be: ['s->3', '3->5', '5->t']
The minimum cost will be: 5.00 
Optimization terminated successfully.


# En este caso, el problema tiene múltiples soluciones, habiendo varias soluciones, cada algoritmo puede elegir una u otra (y lo hace)

# El método simplex no resuelve problemas enteros, que hay respecto a eso?

In [13]:
resume_df['Solution inner_point'] = res.x #.round().astype(int)

In [14]:
resume_df['Solution Simplex'] = res_simplex.x #.round().astype(int)

In [15]:
resume_df

,Min bound,Max bound,Costs,Solution inner_point,Solution Simplex
s->2,0,1,2,5.017178e-01,0.0
s->3,0,1,1,4.982822e-01,1.0
2->4,0,1,2,5.017178e-01,0.0
2->t,0,1,5,5.190454e-11,0.0
3->5,0,1,2,4.982822e-01,1.0
4->t,0,1,1,5.017178e-01,0.0
5->t,0,1,2,4.982822e-01,1.0


In [16]:
if (resume_df['Solution Simplex']!=resume_df['Solution inner_point']).sum()>0:
    print("Las soluciones son diferentes")
else:
    print("Las soluciones son iguales")

if round(res_simplex.fun) == round(res.fun):
    print("Los costos dan iguales: {}".format(res_simplex.fun))
    
else:
    print("Los costos son diferentes")

Las soluciones son diferentes
Los costos dan iguales: 5.0


In [17]:
print('The arcs that make the shortest path trough simplex from s to t will be: %s' % selarcs)

The arcs that make the shortest path trough simplex from s to t will be: ['s->2', '2->4', '4->t']


In [18]:
print("Hay que tener cuidado con la sensibilidad")

Hay que tener cuidado con la sensibilidad


In [19]:
print(res_simplex.message)

Optimization terminated successfully.


# Resume

In [20]:
resume_df.transpose()

,s->2,s->3,2->4,2->t,3->5,4->t,5->t
Min bound,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
Max bound,1.000000,1.000000,1.000000,1.000000e+00,1.000000,1.000000,1.000000
Costs,2.000000,1.000000,2.000000,5.000000e+00,2.000000,1.000000,2.000000
Solution inner_point,0.501718,0.498282,0.501718,5.190454e-11,0.498282,0.501718,0.498282
Solution Simplex,0.000000,1.000000,0.000000,0.000000e+00,1.000000,0.000000,1.000000


In [21]:
print('The arcs that make the shortest path trough inner point from s to t will be: %s' % selarcs)
print('The arcs that make the shortest path trough simplex from s to t will be: %s' % selarcs_simplex)

tol = 1e-3

print("Integer error for method inner point is: ", np.sum(np.abs(res.x.round() - res.x)))

if (np.sum(np.abs(res.x - np.round(res.x))) >= tol):
    print("Warning! Solution is not integer")
    
print("Integer error for method simplex point is: ", np.sum(np.abs(res_simplex.x.round() - res_simplex.x)))

if (np.sum(np.abs(res_simplex.x - np.round(res_simplex.x))) >= tol):
    print("Warning! Solution is not integer")


The arcs that make the shortest path trough inner point from s to t will be: ['s->2', '2->4', '4->t']
The arcs that make the shortest path trough simplex from s to t will be: ['s->3', '3->5', '5->t']
Integer error for method inner point is:  2.9896934005812197
Warning! Solution is not integer
Integer error for method simplex point is:  0.0


## In this case, inner point method doesn't get a perfect integer solution, unlike simplex. Should be careful for sensitivity problems
## Moreover that, simplex method get a different solution than inner point, but the same cost